In [35]:
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [36]:
os.getcwd()

'/Users/juank/Documents/Flatiron/phase_3/chicago_car_crashes/notebooks'

In [59]:
crashes = pd.read_csv('../data/crashes_cleaned.csv')

In [51]:
sorted([x for x in crashes.columns])

['ALIGNMENT_CURVE ON HILLCREST',
 'ALIGNMENT_CURVE, LEVEL',
 'ALIGNMENT_STRAIGHT AND LEVEL',
 'ALIGNMENT_STRAIGHT ON GRADE',
 'ALIGNMENT_STRAIGHT ON HILLCREST',
 'CRASH_DAY_OF_WEEK',
 'CRASH_HOUR',
 'CRASH_MONTH',
 'CRASH_RECORD_ID',
 'CRASH_TYPE',
 'DAMAGE',
 'FIRST_CRASH_TYPE_ANIMAL',
 'FIRST_CRASH_TYPE_FIXED OBJECT',
 'FIRST_CRASH_TYPE_HEAD ON',
 'FIRST_CRASH_TYPE_OTHER NONCOLLISION',
 'FIRST_CRASH_TYPE_OTHER OBJECT',
 'FIRST_CRASH_TYPE_OVERTURNED',
 'FIRST_CRASH_TYPE_PARKED MOTOR VEHICLE',
 'FIRST_CRASH_TYPE_PEDALCYCLIST',
 'FIRST_CRASH_TYPE_PEDESTRIAN',
 'FIRST_CRASH_TYPE_REAR END',
 'FIRST_CRASH_TYPE_REAR TO FRONT',
 'FIRST_CRASH_TYPE_REAR TO REAR',
 'FIRST_CRASH_TYPE_REAR TO SIDE',
 'FIRST_CRASH_TYPE_SIDESWIPE OPPOSITE DIRECTION',
 'FIRST_CRASH_TYPE_SIDESWIPE SAME DIRECTION',
 'FIRST_CRASH_TYPE_TRAIN',
 'FIRST_CRASH_TYPE_TURNING',
 'GEO_KMEANS_Cluster',
 'HIT_AND_RUN_I',
 'INTERSECTION_RELATED_I',
 'LATITUDE',
 'LIGHTING_CONDITION_DARKNESS, LIGHTED ROAD',
 'LIGHTING_CONDITION_DA

In [38]:
target = crashes['PRIM_CONTRIBUTORY_CAUSE']

target_transform = LabelEncoder()

crashes['PRIM_CONTRIBUTORY_CAUSE'] = target_transform.fit_transform(crashes['PRIM_CONTRIBUTORY_CAUSE'])

KeyError: 'WEATHER'

In [39]:
crashes.drop('Unnamed: 0', axis = 1, inplace = True)

# Logistic Regression 

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_validate

In [41]:
X = crashes.loc[:, (crashes.columns != 'PRIM_CONTRIBUTORY_CAUSE') & (crashes.columns != 'CRASH_RECORD_ID')]
y = crashes.loc[:, 'PRIM_CONTRIBUTORY_CAUSE']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.3, random_state = 42)

scaler = StandardScaler()
X_tf_sc = scaler.fit_transform(X_train)

X_test_tr = scaler.transform(X_test)

log_reg = LogisticRegression(solver = 'sag')
log_reg.fit(X_tf_sc, y_train)

LogisticRegression(solver='sag')

In [42]:
pred = log_reg.predict(X_test_tr)
pred_proba = log_reg.predict_proba(X_test_tr)

log_reg.score(X_test_tr, y_test)

0.45730324379309045

In [43]:
log_reg_cv = LogisticRegression(random_state = 42,  solver = 'sag')

cv_results = cross_validate(log_reg_cv, X_train, y_train, cv = 5)

In [44]:
cv_results

{'fit_time': array([175.35853505, 176.66647911, 176.32822824, 176.198488  ,
        175.67583704]),
 'score_time': array([0.05782199, 0.0609839 , 0.06380963, 0.06178999, 0.06492114]),
 'test_score': array([0.4583589 , 0.45784222, 0.45886684, 0.46043305, 0.45793034])}

In [45]:
C_list = [1e-4,1e-3, 1e-2, 1e-1, 1, 10, 100, 1e3]
cv_scores = []
cv_scores_std = []

for c in C_list:
    logreg = LogisticRegression(C = c, solver = 'sag', random_state = 42) 
    cv_loop_results = cross_validate(
                X=X_tf_sc, 
                y=y_train,
                estimator=logreg, 
                cv=5)
    cv_scores.append(np.mean(cv_loop_results['test_score']))
    cv_scores_std.append(np.std(cv_loop_results['test_score']))

KeyboardInterrupt: 